![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and LangChain to make a series of calls to a language model

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate Sequential Chain using langchain integration with watsonx models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The goal of this notebook is to demonstrate how to chain `mistral-small-3-1-24b-instruct-2503` and `mistral-medium-2505` models to generate a sequence of creating a random question on a given topic and an answer to that question and also to make the user friends with LangChain framework, using simple chain (LLMChain) and the extended chain (SequentialChain) with the WatsonxLLM.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [LangChain integration](#watsonxllm)
- [Sequential Chain experiment](#experiment)
- [Python function](#function)
- [Custom inference endpoint](#deployment)
- [Scoring](#scoring)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U "langchain>=0.3,<0.4" | tail -n 1
%pip install -U "langchain_ibm>=0.3,<0.4" | tail -n 1

### Defining the watsonx.ai credentials
This cell defines the watsonx.ai credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [2]:
import getpass

from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your watsonx.ai api key (hit enter): "),
)

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

Create an instance of APIClient with authentication details.

In [4]:
from ibm_watsonx_ai import APIClient

api_client = APIClient(credentials=credentials, project_id=project_id)

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under `TextModels` class.

In [5]:
api_client.foundation_models.TextModels.show()

{'DEFENCE_GRANITE': 'ibm/defence-granite', 'GRANITE_3_2_8B_INSTRUCT': 'ibm/granite-3-2-8b-instruct', 'GRANITE_3_2B_INSTRUCT': 'ibm/granite-3-2b-instruct', 'GRANITE_3_3_8B_INSTRUCT': 'ibm/granite-3-3-8b-instruct', 'GRANITE_3_8B_INSTRUCT': 'ibm/granite-3-8b-instruct', 'GRANITE_4_H_SMALL': 'ibm/granite-4-h-small', 'GRANITE_8B_CODE_INSTRUCT': 'ibm/granite-8b-code-instruct', 'GRANITE_GUARDIAN_3_8B': 'ibm/granite-guardian-3-8b', 'GRANITE_VISION_3_2_2B': 'ibm/granite-vision-3-2-2b', 'LLAMA_3_2_11B_VISION_INSTRUCT': 'meta-llama/llama-3-2-11b-vision-instruct', 'LLAMA_3_2_90B_VISION_INSTRUCT': 'meta-llama/llama-3-2-90b-vision-instruct', 'LLAMA_3_3_70B_INSTRUCT': 'meta-llama/llama-3-3-70b-instruct', 'LLAMA_3_405B_INSTRUCT': 'meta-llama/llama-3-405b-instruct', 'LLAMA_4_MAVERICK_17B_128E_INSTRUCT_FP8': 'meta-llama/llama-4-maverick-17b-128e-instruct-fp8', 'LLAMA_GUARD_3_11B_VISION': 'meta-llama/llama-guard-3-11b-vision', 'MISTRAL_MEDIUM_2505': 'mistralai/mistral-medium-2505', 'MISTRAL_SMALL_3_1_24B_

You need to specify `model_id`'s that will be used for inferencing:

In [6]:
model_id_1 = "mistralai/mistral-small-3-1-24b-instruct-2503"
model_id_2 = "mistralai/mistral-medium-2505"

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to documentation under `GenTextParamsMetaNames` class.

**Action:** If any complications please refer to the <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">documentation</a>.

In [7]:
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1,
}

<a id="watsonxllm"></a>
## LangChain integration

`WatsonxLLM` is a wrapper around watsonx.ai models that provide chain integration around the models.

**Action:** For more details about `CustomLLM` check the <a href="https://python.langchain.com/docs/modules/model_io/models/llms/custom_llm" target="_blank" rel="noopener no referrer">LangChain documentation</a>


### Initialize the `WatsonxLLM` class.

In [8]:
from langchain_ibm import WatsonxLLM

mistral_small_llm = WatsonxLLM(
    model_id=model_id_1,
    url=credentials.url,
    apikey=credentials.api_key,
    project_id=project_id,
    params=parameters,
)
mistral_medium_llm = WatsonxLLM(
    model_id=model_id_2,
    url=credentials.url,
    apikey=credentials.api_key,
    project_id=project_id,
)

You can print all set data about the WatsonxLLM object using the `dict()` method.

In [9]:
mistral_small_llm.dict()

{'model_id': 'mistralai/mistral-small-3-1-24b-instruct-2503',
 'deployment_id': None,
 'params': {'decoding_method': 'sample',
  'max_new_tokens': 100,
  'min_new_tokens': 1,
  'temperature': 0.5,
  'top_k': 50,
  'top_p': 1},
 'project_id': 'eac8bfe2-a00b-43ca-846b-305af5cc6395',
 'space_id': None,
 '_type': 'IBM watsonx.ai'}

<a id="experiment"></a>
## Sequential Chain experiment
The simplest type of sequential chain is called a `SequentialChain`, in which each step has a single input and output and the output of one step serves as the input for the following step.

The experiment will consist in generating a random question about any topic and answer the following question.

An object called `PromptTemplate` assists in generating prompts using a combination of user input, additional non-static data, and a fixed template string.

In our case we would like to create two `PromptTemplate` objects which will be responsible for creating a random question and answering it.

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"],
    template="Generate a random question about {topic}: Question: ",
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

We would like to add functionality around language models using `LLMChain` chain.

`prompt_to_mistral_small` chain formats the prompt template whose task is to generate random question, passes the formatted string to LLM and returns the LLM output.

In [11]:
from langchain.chains import LLMChain

prompt_to_mistral_small = LLMChain(
    llm=mistral_small_llm, prompt=prompt_1, output_key="question"
)

`mistral_medium` chain formats the prompt template whose task is to answer the question we got from `prompt_to_mistral_small` chain, passes the formatted string to LLM and returns the LLM output.

In [12]:
mistral_medium = LLMChain(llm=mistral_medium_llm, prompt=prompt_2, output_key="answer")

This is the overall chain where we run `prompt_to_mistral_small` and `mistral_medium` chains in sequence.

In [13]:
from langchain.chains import SequentialChain

qa = SequentialChain(
    chains=[prompt_to_mistral_small, mistral_medium],
    input_variables=["topic"],
    output_variables=["question", "answer"],
    verbose=True,
)

Generate random question and answer to topic.

In [14]:
qa.invoke({"topic": "life"})



> Entering new SequentialChain chain...

> Finished chain.


{'topic': 'life',
 'question': ' If you could have any superpower that would help improve the world, what would it be and how would you use it?\n\nAnswer: If I could have any superpower to improve the world, I would choose the ability to communicate with and understand all living beings. This power would allow me to bridge the gap between humans and other species, fostering a deeper understanding and empathy for the natural world. I would use this ability to advocate for better conservation practices, reduce conflicts between humans and wildlife, and',
 'answer': ' promote a more harmonious relationship between all forms of life. Additionally, understanding the needs and languages of'}

<a id="function"></a>
## Python function
Let's wrap the chain code within python function that can be used as a content of inference endpoint.

### Function implementation.
Let's wrap the above chain code into function.

In [15]:
ai_params = {
    "url": credentials.url,
    "apikey": credentials.api_key,
    "project_id": project_id,
    "generation_parameters": parameters,
}

In [16]:
def chain_text_generator(params=ai_params):

    from langchain import PromptTemplate
    from langchain.chains import LLMChain, SequentialChain
    from langchain_ibm import WatsonxLLM

    url = params["url"]
    apikey = params["apikey"]
    project_id = params["project_id"]
    parameters = params["generation_parameters"]
    mistral_small_llm = WatsonxLLM(
        model_id="mistralai/mistral-small-3-1-24b-instruct-2503",
        url=url,
        apikey=apikey,
        project_id=project_id,
        params=parameters,
    )
    mistral_medium_llm = WatsonxLLM(
        model_id="mistralai/mistral-medium-2505",
        url=url,
        apikey=apikey,
        project_id=project_id,
    )
    prompt_1 = PromptTemplate(
        input_variables=["topic"],
        template="Generate a random question about {topic}: Question: ",
    )
    prompt_2 = PromptTemplate(
        input_variables=["question"],
        template="Answer the following question: {question}",
    )
    prompt_to_mistral_small = LLMChain(
        llm=mistral_small_llm, prompt=prompt_1, output_key="question"
    )
    mistral_medium = LLMChain(
        llm=mistral_medium_llm, prompt=prompt_2, output_key="answer"
    )
    chain = SequentialChain(
        chains=[prompt_to_mistral_small, mistral_medium],
        input_variables=["topic"],
        output_variables=["question", "answer"],
    )

    def score(payload):
        """Generates question based on provided topic and returns the answer."""

        answer = chain.invoke({"topic": payload["input_data"][0]["values"][0][0]})
        return {
            "predictions": [
                {
                    "fields": ["topic", "question", "answer"],
                    "values": [answer["topic"], answer["question"], answer["answer"]],
                }
            ]
        }

    return score

### Test the function
It is good practice to validate the code locally first.

In [17]:
sample_payload = {"input_data": [{"fields": ["topic"], "values": [["life"]]}]}

inference = chain_text_generator()
inference(sample_payload)

{'predictions': [{'fields': ['topic', 'question', 'answer'],
   'values': ['life',
    ' "If you could have any superpower to make the world a better place, what would it be and why?"\n\nThis question encourages reflection on personal values and the desire to make a positive impact on the world. It also allows for creative and imaginative responses, making it an engaging topic for discussion. Here are a few possible answers to get the conversation started:\n\n1. **Teleportation**: "I would choose teleportation so I could instantly travel to places in need of help, whether it',
    '\'s delivering food to a starving village or evacuating people from a natural disaster."\n2. **']}]}

<a id="deployment"></a>
## Custom inference endpoint
Create the online deployment of python function.

### Custom software specification
Create new software specification based on default Python 3.11 environment extended by langchain package.

In [18]:
config_yml = """
name: python311
channels:
  - empty
dependencies:
  - pip:
    - langchain_ibm==0.3.3
    - langchain==0.3.7
prefix: /opt/anaconda3/envs/python311
"""

with open("config.yaml", "w", encoding="utf-8") as f:
    f.write(config_yml)

In [19]:
space_id = "PASTE YOUR SPACE ID HERE"

Now you need to store new package extension with APIClient.

In [20]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)
client.set.default_space(space_id)
base_sw_spec_id = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "langchain watsonx.ai env",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Environment with langchain",
    client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml",
}

pkg_extn_details = client.package_extensions.store(
    meta_props=meta_prop_pkg_extn, file_path="config.yaml"
)
pkg_extn_id = client.package_extensions.get_id(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)

Creating package extension
SUCCESS


Create new software specification and add created package extension to it.

In [21]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "langchain watsonx.ai custom software specification",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for statsmodels",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
        "guid": base_sw_spec_id
    },
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = client.software_specifications.get_id(sw_spec_details)
client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)

SUCCESS


'SUCCESS'

### Store the function

In [22]:
meta_props = {
    client.repository.FunctionMetaNames.NAME: "SequenceChain LLM function",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: sw_spec_id,
}

function_details = client.repository.store_function(
    meta_props=meta_props, function=chain_text_generator
)
function_id = client.repository.get_function_id(function_details)

### Create online deployment

In [23]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of LLMs chain function",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
}

function_deployment = client.deployments.create(function_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: '268dfb43-78af-426e-8800-ac34618b76f9' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
..........
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='5f818c96-7c44-4ab7-9867-02c71f245352'
-----------------------------------------------------------------------------------------------




<a id="scoring"></a>
## Scoring
Generate text using custom inference endpoint.

In [24]:
deployment_id = client.deployments.get_id(function_deployment)
client.deployments.score(deployment_id, sample_payload)

{'predictions': [{'fields': ['topic', 'question', 'answer'],
   'values': ['life',
    " If you could live in any era of history, which one would you choose and why?\n\nThis question prompts reflection on personal values, interests, and historical periods that resonate with the individual. It encourages a deeper understanding of how different eras might align with one's aspirations, curiosities, or desired lifestyles. For example, someone might choose the Renaissance for its artistic and intellectual flourishing, while another might prefer the 1960s for its cultural and social movements.",
    " The reasons behind the choice can reveal a lot about a person's personality, including their appreciation for certain"]}]}

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!
 
 You learned how to use Sequential Chain using custom llm WastonxLLM.
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors:
 **Lukasz Cmielowski, PhD**, Senior Technical Staff Member at watsonx.ai.
 
 **Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2024-2026 IBM. This notebook and its source code are released under the terms of the MIT License.